In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
avl = pd.read_csv('GMU_2021_Summer_AVL.csv')
avl.columns = ['AVL_IncidentNumber',
               'AVL_UnitID',
               'AVL_Timestamp',
               'AVL_UnitStatus',
               'AVL_Speed']
avl.dropna(subset = ['AVL_IncidentNumber'],
           inplace = True)
avl.reset_index(inplace = True, drop = True)
avl.isna().sum()

AVL_IncidentNumber    0
AVL_UnitID            0
AVL_Timestamp         0
AVL_UnitStatus        0
AVL_Speed             0
dtype: int64

In [3]:
# convert object to timestamp proper
avl['AVL_Timestamp'] = avl['AVL_Timestamp'].str.replace('ED', '')
avl['AVL_Timestamp'] = avl['AVL_Timestamp'].str.replace('ES', '')
avl['AVL_Timestamp'] = pd.to_datetime(avl['AVL_Timestamp'], format = '%Y%m%d%H%M%S')

In [5]:
avl['AVL_FullKey'] = avl['AVL_IncidentNumber'].map(str) +  avl['AVL_UnitID'].map(str) +  avl['AVL_UnitStatus'].map(str)

In [9]:
avl = avl.sort_values(by='AVL_Timestamp', ascending=True)
avl_CLEAN = avl.drop_duplicates(subset='AVL_FullKey', keep="first", inplace=False)

In [30]:
avl_ER = avl_CLEAN[(avl_CLEAN["AVL_UnitStatus"] == "ER")]

In [27]:
notMoving_ER = avl_ER[(avl_ER["AVL_Speed"] == "not moving")]

In [28]:
notMoving_ER.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208134 entries, 3849903 to 17911943
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   AVL_IncidentNumber  208134 non-null  object        
 1   AVL_UnitID          208134 non-null  object        
 2   AVL_Timestamp       208134 non-null  datetime64[ns]
 3   AVL_UnitStatus      208134 non-null  object        
 4   AVL_Speed           208134 non-null  object        
 5   AVL_FullKey         208134 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 11.1+ MB


In [29]:
208134/711508

0.29252517188843974

In [16]:
cad = pd.read_csv('GMU_2021_Summer_CAD.csv')
cad['CAD_Timestamp'] = cad['CAD_Timestamp'].str.replace('ED', '')
cad['CAD_Timestamp'] = cad['CAD_Timestamp'].str.replace('ES', '')
cad['CAD_Timestamp'] = pd.to_datetime(cad['CAD_Timestamp'], format = '%Y%m%d%H%M%S')
cad['CAD_FullKey'] = cad['CAD_IncidentNumber'].map(str) +  cad['CAD_UnitID'].map(str) +  cad['CAD_UnitStatus'].map(str)
cad = cad.sort_values(by='CAD_Timestamp', ascending=True)
cad_CLEAN = cad.drop_duplicates(subset='CAD_FullKey', keep="first", inplace=False)

In [31]:
cad_ER = cad_CLEAN[(cad_CLEAN["CAD_UnitStatus"] == "ER")]

In [32]:
cad_ER.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 822765 entries, 3862128 to 1702468
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   CAD_IncidentNumber  822765 non-null  object        
 1   CAD_UnitID          822765 non-null  object        
 2   CAD_Timestamp       822765 non-null  datetime64[ns]
 3   CAD_UnitStatus      822765 non-null  object        
 4   CAD_FullKey         822765 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 37.7+ MB


In [22]:
cad_CLEAN.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4186845 entries, 2515485 to 1702468
Data columns (total 5 columns):
 #   Column              Dtype         
---  ------              -----         
 0   CAD_IncidentNumber  object        
 1   CAD_UnitID          object        
 2   CAD_Timestamp       datetime64[ns]
 3   CAD_UnitStatus      object        
 4   CAD_FullKey         object        
dtypes: datetime64[ns](1), object(4)
memory usage: 191.7+ MB


In [34]:
df_er = cad_ER.merge(avl_CLEAN, 
               how = 'inner', 
               left_on = ['CAD_IncidentNumber',
                          'CAD_UnitID',
                          'CAD_UnitStatus'],
               right_on = ['AVL_IncidentNumber',
                           'AVL_UnitID',
                           'AVL_UnitStatus'],
               suffixes = ('_avl', '_cad'))

In [35]:
df_er.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 711500 entries, 0 to 711499
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   CAD_IncidentNumber  711500 non-null  object        
 1   CAD_UnitID          711500 non-null  object        
 2   CAD_Timestamp       711500 non-null  datetime64[ns]
 3   CAD_UnitStatus      711500 non-null  object        
 4   CAD_FullKey         711500 non-null  object        
 5   AVL_IncidentNumber  711500 non-null  object        
 6   AVL_UnitID          711500 non-null  object        
 7   AVL_Timestamp       711500 non-null  datetime64[ns]
 8   AVL_UnitStatus      711500 non-null  object        
 9   AVL_Speed           711500 non-null  object        
 10  AVL_FullKey         711500 non-null  object        
dtypes: datetime64[ns](2), object(9)
memory usage: 65.1+ MB


In [36]:
notMoving_cadER = df_er[(df_er["CAD_UnitStatus"] == "ER") & (df_er["AVL_Speed"] == "not moving")]

In [37]:
notMoving_cadER.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208132 entries, 6 to 711499
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   CAD_IncidentNumber  208132 non-null  object        
 1   CAD_UnitID          208132 non-null  object        
 2   CAD_Timestamp       208132 non-null  datetime64[ns]
 3   CAD_UnitStatus      208132 non-null  object        
 4   CAD_FullKey         208132 non-null  object        
 5   AVL_IncidentNumber  208132 non-null  object        
 6   AVL_UnitID          208132 non-null  object        
 7   AVL_Timestamp       208132 non-null  datetime64[ns]
 8   AVL_UnitStatus      208132 non-null  object        
 9   AVL_Speed           208132 non-null  object        
 10  AVL_FullKey         208132 non-null  object        
dtypes: datetime64[ns](2), object(9)
memory usage: 19.1+ MB


In [38]:
208132/711500

0.29252565003513703